### Create Step Function

Now we have to create a step function to orchestrate the execution of all the previous step. 

For step function we use the ASL markup language. 

This is the resulting diagram:

<img src='steps.png'></img>

In [1]:
! cat stepFunction.asl

{
  "StartAt": "ETL",
  "States": {
    "ETL": {
        "Type": "Task",
        "Resource": "arn:aws:states:::glue:startJobRun.sync",
        "Parameters": {
          "JobName": "etlandpipeline"
        },
        "Next": "StartTrainingJob"
      },
      "StartTrainingJob": {
              "Type": "Task",
              "Resource": "arn:aws:lambda:us-east-1:452432741922:function:lambdaModelTrain",
              "ResultPath": "$",
              "Next": "CheckStatusTraining"
            },
            "CheckStatusTraining": {
              "Type": "Task",
              "Resource": "arn:aws:lambda:us-east-1:452432741922:function:lambdaModelAwait",
              "ResultPath": "$",
              "Next": "CheckTrainingBranch"
            },
        "CheckTrainingBranch": {
              "Type": "Choice",
              "Choices": [
                {
                  "Or": [{
                      "Variable": "$.status",
                      "StringEquals": "Completed"
                    